In [1]:
import numpy as np
import pylidc as pl
from sklearn.model_selection import train_test_split
from unet import UNet

Using TensorFlow backend.


In [2]:
scans = pl.query(pl.Scan)
n = scans.count()

In [3]:
image_size = (256, 256)

In [ ]:
def get_data(i):
    scan = scans[i]
    images = scan.load_all_dicom_images(verbose=False)
    n = len(images)
    X = np.array([resize(im.pixel_array, image_size, mode='constant') for im in images])
    X = np.expand_dims(X, axis=-1)
    y = np.zeros((512, 512, n))
    for ann in scan.annotations:
        b = ann.bbox()
        y[b] = np.logical_or(y[b], ann.boolean_mask())
    y = np.rollaxis(y, 2, 0)
    y = resize(y, (n, *image_size), mode='constant')
    if only_nodules:
        indices = [l.sum() > 0 for l in y]
        X = X[indices]
        y = y[indices]
    y = np.expand_dims(y, axis=-1)
    return X, y